# Valuing a Swap on Leg Level

This notebook will upsert an example swap and then retrieve both legs individually and value them. This will allow you to utilize our swap model when you need to have each leg valued individually for reporting reasons.

## Table of Contents:
- 1. [Swap Creation](#1.-Swap-Creation)
- 2. [Valuation](#2.-Valuation)
- 3. [Booking Legs using Fund Accounting Data](#3.-Booking-Legs-using-Fund-Accounting-Data)
- 4. [Finding the Parent Swap](#4.-Finding-the-Parent-Swap)

In [122]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.cocoon.cocoon import load_from_data_frame

from lusidtools.cocoon.cocoon_printer import format_instruments_response

# Import Libraries
from datetime import datetime, timedelta
from lusidtools.lpt.lpt import to_date
from IPython.display import display_html
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import json
import os

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)


LUSID Environment Initialised
LUSID SDK Version:  0.6.11051.0


In [123]:
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
complex_market_data_api = api_factory.build(lu.api.ComplexMarketDataApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
relationship_definitions_api = api_factory.build(lu.api.RelationshipDefinitionsApi)
relationships_api = api_factory.build(lu.api.RelationshipsApi)

In [124]:
instrument_scope = "legLevelSwaps"
scope = "legLevelSwaps"

# 1. Swap Creation

In this section, we will create the swap using the data outlined belowL. In the next section, we will book it in a portfolio and value it.

In [125]:
# Set swap details
trade_date = datetime(2018, 2, 22, 0, 0, tzinfo=pytz.utc).isoformat()
start_date = trade_date
end_date = datetime(2023, 2, 16, 0, 0, tzinfo=pytz.utc).isoformat()
instrument_ccy = "USD"
payment_frequency = "1M"
index_tenor = "1M"
reference_index = "LIBOR"
cpn = 0.025
spread = 0.02
day_count_convention = "Actual360"
stub_type = "None"
name = "HSBS SWAP IRS 2/23 LIBOR"
instrument_identifier = "CJA023000"

# Set the valuation date
effective_at = datetime(2022, 10, 27, 1, 0, tzinfo=pytz.utc).isoformat()

## Fixed Leg
We will first be setting up the fixed leg of the swap.

In [126]:
# Create the fixed leg details
fixed_flows = lm.FlowConventions(
    currency=instrument_ccy,
    payment_frequency=payment_frequency,
    day_count_convention=day_count_convention,
    roll_convention="ModifiedFollowing",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0
)

fixed_leg_definition = lm.LegDefinition(
    rate_or_spread=cpn,
    pay_receive="Receive",
    conventions=lm.FlowConventions(
        currency=instrument_ccy,
        payment_frequency=payment_frequency,
        day_count_convention=day_count_convention,
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0
    ),
    stub_type=stub_type,
    notional_exchange_type="None"
)

# Assemble the fixed leg
fixed_leg = lm.FixedLeg(
    start_date=start_date,
    maturity_date=end_date,
    # Unitized notional, could also be passed directly as 10MM
    notional=1,
    leg_definition=fixed_leg_definition,
    instrument_type="FixedLeg"
)

## Floating Leg

Now we book in the floating leg of the swap.

In [127]:
# Create the floating leg details
floating_leg_definition = lm.LegDefinition(
    rate_or_spread = spread,
    index_convention = lm.IndexConvention(
        currency=instrument_ccy,
        payment_tenor=index_tenor,
        fixing_reference=reference_index,
        publication_day_lag=0,
        day_count_convention=day_count_convention,
        index_name=reference_index
    ),
    pay_receive="Pay",
    conventions=lm.FlowConventions(
        currency=instrument_ccy,
        payment_frequency=payment_frequency,
        day_count_convention=day_count_convention,
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0
    ),
    stub_type=stub_type,
    notional_exchange_type="None",
    reset_convention="InArrears"
)

# Assemble the floating leg
floating_leg = lm.FloatingLeg(
    start_date=start_date,
    maturity_date=end_date,
    # Unitized notional, could also be passed directly
    notional=1,
    leg_definition=floating_leg_definition,
    instrument_type="FloatingLeg"
)

## Swap instrument

Finally, we assmble both legs into the swap instrument.

In [128]:
# Store legs in a list
irs_legs = [
    fixed_leg,
    floating_leg
]

# Create the swap
instrument_definition = lm.InterestRateSwap(
    start_date=start_date,
    maturity_date=end_date,
    legs=irs_legs,
    instrument_type="InterestRateSwap"
)

# Create instrument definition
swap_instrument_definition = lm.InstrumentDefinition(
    name=name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=instrument_identifier)},
    definition=instrument_definition
)
upsert_request = {instrument_identifier: swap_instrument_definition}

# Upsert instrument to LUSID
response = instruments_api.upsert_instruments(
    request_body=upsert_request,
    scope=instrument_scope
)

if len(response.failed) == 0:
    print(f"Instrument {swap_instrument_definition.name} was successfully upserted into LUSID")
else:
    print("An error occurred with the above upsert_instruments call, see error message:", response.failed)

Instrument HSBS SWAP IRS 2/23 LIBOR was successfully upserted into LUSID


## Display both legs of the swap
Let's start by backing out both leg definitions of the swap we just created.

In [129]:
# Define helper tools
def get_instrument(
        identifier: str,
        identifier_type: str="ClientInternal",
        scope: str=None,
   )-> lm.Instrument:
    return instruments_api.get_instrument(
        identifier_type=identifier_type,
        identifier=identifier,
        scope=scope
    )

# Get and store the instrument response
response = get_instrument(identifier=instrument_identifier, scope=instrument_scope)

# Store LUID
luid = response.lusid_instrument_id

# Store response leg definitions as dataframes
flt_leg_df = lusid_response_to_data_frame(response.instrument_definition.legs[1])
fix_leg_df = lusid_response_to_data_frame(response.instrument_definition.legs[0])

# Display the leg dataframes
df1_styler = flt_leg_df.style.set_table_attributes("style='display:inline'").set_caption('Floating Leg')
df2_styler = fix_leg_df.style.set_table_attributes("style='display:inline'").set_caption('Fixed Leg')
display_html(df1_styler._repr_html_() + df2_styler._repr_html_(), raw=True)

,response_values
start_date,2018-02-22 00:00:00+00:00
maturity_date,2023-02-16 00:00:00+00:00
leg_definition.convention_name,None
leg_definition.conventions.currency,USD
leg_definition.conventions.payment_frequency,1M
leg_definition.conventions.day_count_convention,Actual360
leg_definition.conventions.roll_convention,ModifiedFollowing
leg_definition.conventions.payment_calendars,[]
leg_definition.conventions.reset_calendars,[]
leg_definition.conventions.settle_days,0


# 2. Valuation

## Portfolio

We will now create a portfolio and book a transaction so that it holds our swap. This way we can value it later into the notebook.

In [130]:
# Setup scope and code for the portfolio
portfolio_scope = scope
portfolio_code = "LegLevelSwapsPortfolio"

# Create portfolio
try:
    transaction_portfolios_api.create_portfolio(
        scope=portfolio_scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            instrument_scopes=[instrument_scope]
        ),
    )
    print(f"Portfolio for {portfolio_scope} | {portfolio_code} created successfully.")

except lu.ApiException as e:
    print(json.loads(e.body)["title"])


Could not create a portfolio with id 'LegLevelSwapsPortfolio' because it already exists in scope 'legLevelSwaps'.


In [131]:
trade = lm.TransactionRequest(
    transaction_id="TXN001",
    type="Buy",
    instrument_identifiers={
        "Instrument/default/ClientInternal": f"{instrument_identifier}"
    },
    transaction_date=trade_date,
    settlement_date=trade_date,
    units=1000000,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency="USD"),
    transaction_currency="USD"
)


response = transaction_portfolios_api.upsert_transactions(
    scope=portfolio_scope,
    code=portfolio_code,
    transaction_request=[trade]
)

print(f"Transaction updated for effective time: {response.version.effective_from}.")

Transaction updated for effective time: 2010-01-01 00:00:00+00:00.


## Quotes

We upsert quotes for 27/10/2022, which is the date we will value our swap on.

In [132]:
quotes = {'client_internal': [instrument_identifier],
          'date': effective_at,
          'price': [1],
          'currency': ['USD']}
quotes_df = pd.DataFrame(data=quotes)
quotes_df

,client_internal,date,price,currency
0,CJA023000,2022-10-27T01:00:00+00:00,1,USD


In [133]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["client_internal"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=row["date"],
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
        scale_factor=100,
    )
    for index, row in quotes_df.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quote successfully loaded into LUSID. 1 quotes loaded.


## Upsert Recipe

We upsert a recipe that uses the ConstantTimeValueOfMoney model so that we can see the valuation of the swap.
To return the legs individually, we set the produce_separate_result_for_linear_otc_legs parameter to True.

In [134]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="swapValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Credit.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="InterestRateSwap",
                parameters="{}",
            )
        ],
        options=lm.PricingOptions(
            produce_separate_result_for_linear_otc_legs=True
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## Valuation

Now we create a valuation function using the aggregation api and value our swap.

In [135]:
def get_val(effective_at, portfolio_code, recipe_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Valuation/Accrued/Amount","Value")
        ],
        #group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_at),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Valuation/Accrued/Amount": "Accrued Interest",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

In [136]:
get_val(effective_at, portfolio_code, "swapValuation")

,Present Value,Accrued Interest,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units
0,"6,319.4444",208.3333,HSBS SWAP IRS 2/23 LIBOR,CJA023000,None,"1,000,000.0000"
1,"-5,055.5556",-166.6667,HSBS SWAP IRS 2/23 LIBOR,CJA023000,None,"1,000,000.0000"


You can see this within the LUSID UI on the Returns Dashboard by navigating to the following link:

In [147]:
api_url = os.getenv("FBN_LUSID_API_URL")
url = f"{api_url[:-3]}app/dashboard/valuations?scope=legLevelSwaps&code=LegLevelSwapsPortfolio&entityType=Portfolio&recipeScope=legLevelSwaps&recipeCode=swapValuation&effectiveDate=2022-10-27T01:00:00.000Z"
print(url)



https://lorenz.lusid.com/app/dashboard/valuations?scope=legLevelSwaps&code=LegLevelSwapsPortfolio&entityType=Portfolio&recipeScope=legLevelSwaps&recipeCode=swapValuation&effectiveDate=2022-10-27T01:00:00.000Z


### Single Line Valuation

To show the difference, we will now value the swap as one instrument. We expect the PV to be the difference of both legs (6,319.44 - 5,055.55 = 1,263.89).

We will first create a new recipe based on the previous one, but with produce_separate_result_for_linear_otc_legs set to False.

In [138]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="swapValuationNoSplit",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Credit.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="InterestRateSwap",
                parameters="{}",
            )
        ],
        options=lm.PricingOptions(
            produce_separate_result_for_linear_otc_legs=False
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

In [139]:
get_val(effective_at, portfolio_code, "swapValuationNoSplit")

,Present Value,Accrued Interest,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units
0,"1,263.8889",41.6667,HSBS SWAP IRS 2/23 LIBOR,CJA023000,None,"1,000,000.0000"


You can see this as well within the LUSID UI on the Returns Dashboard by navigating to the following link:

In [148]:
api_url = os.getenv("FBN_LUSID_API_URL")
url = f"{api_url[:-3]}app/dashboard/valuations?scope=legLevelSwaps&code=LegLevelSwapsPortfolio&entityType=Portfolio&recipeScope=legLevelSwaps&recipeCode=swapValuationNoSplit&effectiveDate=2022-10-27T01:00:00.000Z"
print(url)

https://lorenz.lusid.com/app/dashboard/valuations?scope=legLevelSwaps&code=LegLevelSwapsPortfolio&entityType=Portfolio&recipeScope=legLevelSwaps&recipeCode=swapValuationNoSplit&effectiveDate=2022-10-27T01:00:00.000Z
